# Homework: Neural Network

### Import Libraries

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Model

In [26]:
def make_model(input_size=150, learning_rate=0.002, size_inner=64):

    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(input_size, input_size, 3))(inputs)

    pool = keras.layers.MaxPooling2D(pool_size=(2,2))(base)

    vectors = keras.layers.Flatten()(pool)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    
    outputs = keras.layers.Dense(1, activation='sigmoid')(inner)
    
    model = keras.Model(inputs, outputs)
    
    #########################################

    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

#### Question1: what is the best loss function for Binay Classification? 
Ans---> Binary Crossentropy

#### Question 2: What's the total number of parameters of the model?

In [36]:
model = make_model()
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 175232)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                11214912  
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
_______________________________________________

## Generators and Training

In [37]:
path = os.path.join(os.getcwd(), 'Dataset_Homework')
path_train = os.path.join(path, 'Train')
path_val = os.path.join(path, 'Validation')

In [38]:
train_gen = ImageDataGenerator(rescale=1./255)
train_ds = train_gen.flow_from_directory(path_train, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)

Found 20000 images belonging to 2 classes.


In [39]:
val_gen = ImageDataGenerator(rescale=1./255)
val_ds = val_gen.flow_from_directory(path_val, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)

Found 5000 images belonging to 2 classes.


In [42]:
history = model.fit(train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 62s 598ms/step - loss: 0.6938 - accuracy: 0.5310 - val_loss: 0.6851 - val_accuracy: 0.5290
Epoch 2/10
100/100 [==============================] - 54s 542ms/step - loss: 0.6870 - accuracy: 0.5495 - val_loss: 0.6888 - val_accuracy: 0.5300
Epoch 3/10
100/100 [==============================] - 55s 550ms/step - loss: 0.6773 - accuracy: 0.5690 - val_loss: 0.6786 - val_accuracy: 0.5800
Epoch 4/10
100/100 [==============================] - 52s 523ms/step - loss: 0.6719 - accuracy: 0.5810 - val_loss: 0.6719 - val_accuracy: 0.5960
Epoch 5/10
100/100 [==============================] - 59s 591ms/step - loss: 0.6702 - accuracy: 0.5875 - val_loss: 0.6639 - val_accuracy: 0.5830
Epoch 6/10
100/100 [==============================] - 56s 561ms/step - loss: 0.6669 - accuracy: 0.5930 - val_loss: 0.6696 - val_accuracy: 0.5710
Epoch 7/10
100/100 [==============================] - 54s 543ms/step - loss: 0.6489 - accuracy: 0.6125 - val_loss: 0.6394 - val_ac

In [43]:
history.history

{'loss': [0.6937652230262756,
  0.6870418787002563,
  0.6773014664649963,
  0.6719407439231873,
  0.6702414751052856,
  0.6669207215309143,
  0.6488724946975708,
  0.6507542729377747,
  0.6486923098564148,
  0.6366235017776489],
 'accuracy': [0.531000018119812,
  0.5494999885559082,
  0.5690000057220459,
  0.5809999704360962,
  0.5874999761581421,
  0.5929999947547913,
  0.612500011920929,
  0.6100000143051147,
  0.6134999990463257,
  0.6349999904632568],
 'val_loss': [0.6850662231445312,
  0.6888059377670288,
  0.6786280274391174,
  0.671902596950531,
  0.6639358997344971,
  0.6696205735206604,
  0.6393505334854126,
  0.6405614018440247,
  0.6450397968292236,
  0.6184349656105042],
 'val_accuracy': [0.5289999842643738,
  0.5299999713897705,
  0.5799999833106995,
  0.5960000157356262,
  0.5830000042915344,
  0.5709999799728394,
  0.6230000257492065,
  0.6370000243186951,
  0.6330000162124634,
  0.6600000262260437]}

#### Question 3: What is the median of training accuracy for this model?

In [48]:
median_train_accuracy = np.median(history.history['accuracy'])
round(median_train_accuracy, 2)

0.59

#### Question4: What is the standard deviation of training loss for this model?

In [50]:
std_train_loss = np.std(history.history['loss'])
round(std_train_loss, 3)

0.018

## Data Augmentation

In [51]:
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_ds = train_gen.flow_from_directory(path_train, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)

Found 20000 images belonging to 2 classes.


In [52]:
val_gen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
val_ds = val_gen.flow_from_directory(path_val, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)

Found 5000 images belonging to 2 classes.


In [54]:
history = model.fit(train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50)

Epoch 1/10
100/100 [==============================] - 83s 833ms/step - loss: 0.6655 - accuracy: 0.5900 - val_loss: 0.6499 - val_accuracy: 0.6260
Epoch 2/10
100/100 [==============================] - 79s 790ms/step - loss: 0.6597 - accuracy: 0.6160 - val_loss: 0.6597 - val_accuracy: 0.6080
Epoch 3/10
100/100 [==============================] - 77s 773ms/step - loss: 0.6668 - accuracy: 0.5750 - val_loss: 0.6529 - val_accuracy: 0.6350
Epoch 4/10
100/100 [==============================] - 79s 793ms/step - loss: 0.6555 - accuracy: 0.6110 - val_loss: 0.6530 - val_accuracy: 0.6210
Epoch 5/10
100/100 [==============================] - 81s 813ms/step - loss: 0.6578 - accuracy: 0.6120 - val_loss: 0.6570 - val_accuracy: 0.6090
Epoch 6/10
100/100 [==============================] - 83s 830ms/step - loss: 0.6563 - accuracy: 0.5995 - val_loss: 0.6610 - val_accuracy: 0.6130
Epoch 7/10
100/100 [==============================] - 75s 751ms/step - loss: 0.6478 - accuracy: 0.6130 - val_loss: 0.6435 - val_ac

#### Question 5: What is the mean of validation loss for the model trained with augmentations?

In [56]:
mean_val_loss = np.mean(history.history['val_loss'])
round(mean_val_loss, 2)

0.65

#### Question 6: What's the average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [61]:
mean_val_acc = np.mean(history.history['val_accuracy'][5:])
round(mean_val_acc, 2)

0.61